In [1]:
from webdriver_manager.chrome import ChromeDriverManager # 자동으로 크롬드라이버(가상브라우저) 파일을 다운로드해주는 라이브러리
from selenium.webdriver.chrome.service import Service # 다운로드된 크롬드라이버 파일을 연결하기 위해 활용

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup 
import time
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
df_movies = pd.read_excel('movie_raw_data.xlsx')
df_movies

,왓챠피디아 평점 TOP 영화,링크
0,하이큐!! 땅 VS 하늘,https://pedia.watcha.com/ko-KR/contents/mO1j3DL
1,대부 일대기,https://pedia.watcha.com/ko-KR/contents/md6B3Ad
2,쇼생크 탈출,https://pedia.watcha.com/ko-KR/contents/mNdKBkO
3,인생은 아름다워,https://pedia.watcha.com/ko-KR/contents/mV539wO
4,하이큐!! 콘셉트의 싸움,https://pedia.watcha.com/ko-KR/contents/mW4LYBo
...,...,...
295,코메디의 왕,https://pedia.watcha.com/ko-KR/contents/mV53EwO
296,바람,https://pedia.watcha.com/ko-KR/contents/mLOP7lW
297,몸 값,https://pedia.watcha.com/ko-KR/contents/m533xZ3
298,중앙역,https://pedia.watcha.com/ko-KR/contents/mgOAvld


In [3]:
df_users = pd.DataFrame(columns = ['유저','유저 링크'])

In [ ]:
service = Service(executable_path=ChromeDriverManager().install()) 
driver = webdriver.Chrome(service=service)

# 영화 각각 링크로 들어가기
for i in range (0,300):
    df_movies_links = df_movies['링크'][i] + '/comments'
    driver.get(df_movies_links) #여기서 반복문

    # 광고 지우기
    time.sleep(2)
    close_button = '/html/body/div/div/div[2]/div/div/div/button/span'
    driver.find_element_by_xpath(close_button).click()
    
    # 스크롤 내리기 (4초; 생각보다 코멘트 로딩이 느림 / 스크롤이 너무 빨리 내려가면 더이상 안 내려가는 오류발생)
    time.sleep(2)
    while True:
        lastHeight = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)
        newHeight = driver.execute_script("return document.body.scrollHeight")
        if newHeight == lastHeight:
             break
    
    # Beautifulsoup으로 유저 데이터
    user_name = []
    review_url = driver.current_url
    movie_url = 'https://pedia.watcha.com'
    user_link = []
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    users = soup.find_all('div',{'class':'css-1cvf9dk'})

    for user in users:
        user_name.append(user.find('a').get('title'))
        user_link.append(movie_url + user.find('a').get('href'))

    df_user = pd.DataFrame(zip(user_name,user_link),columns=['유저','유저 링크'])
    df_users = df_users.append(df_user)

    # 유저 데이터 행 구하기; 링크 index용
    num_user = df_user.shape[0]

# 드라이버 끝내기
driver.close()
driver.quit()

df_users



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\mandu\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


In [24]:
# 드라이버 끝내기
driver.close()
driver.quit()

In [ ]:
df_users.to_excel('users_raw_data.xlsx', index = False, encoding = 'utf-8-sig')